In [8]:
import torch
from src import args as arg
from src import utils
from src.models.model import IRNet
from src.rule import semQL
import argparse
import sem2SQL
import copy
import os
import json
import nltk
import re
import tempfile
import shutil

args = argparse.Namespace(
    action_embed_size=128,
    att_vec_size=300,
    batch_size=64,
    beam_size=10,
    clip_grad=5.0,
    col_embed_size=300,
    column_att='affine',
    column_pointer=True,
    cuda=True,
    dataset='./fdata',
    decode_max_time_step=40,
    dropout=0.3,
    embed_size=300,
    epoch=50,
    glove_embed_path='./data/glove.42B.300d.txt',
    hidden_size=300,
    load_model='./saved_model/IRNet_pretrained.model',
    loss_epoch_threshold=50,
    lr=0.001,
    lr_scheduler=True,
    lr_scheduler_gammar=0.5,
    lstm='lstm',
    max_epoch=-1,
    model_name='rnn',
    no_query_vec_to_action_map=False,
    optimizer='Adam',
    query_vec_to_action_diff_map=False,
    readout='identity',
    save='save',
    save_to='model',
    seed=90,
    sentence_features=True,
    sketch_loss_coefficient=1.0,
    toy=False,
    type_embed_size=128,
    word_dropout=0.2
)

def load_data(tmpdir):
    global table_data, sql_data, schemas
    
    import copy
    cargs = copy.deepcopy(args)
    cargs.dataset = os.path.join(tmpdir, 'fdata')
    
    _, table_data, sql_data, _ = utils.load_dataset(cargs.dataset, use_small=args.toy)
    with open(os.path.join(args.dataset, 'tables.json'), 'r', encoding='utf8') as f:
        table_datas = json.load(f)
    schemas = dict()
    for i in range(len(table_datas)):
        schemas[table_datas[i]['db_id']] = table_datas[i]

        
def load_model():
    global model
    
    grammar = semQL.Grammar()
    model = IRNet(args, grammar)
    if args.cuda: model.cuda()
    print('load pretrained model from %s'% (args.load_model))
    pretrained_model = torch.load(args.load_model,
                                     map_location=lambda storage, loc: storage)
    pretrained_modeled = copy.deepcopy(pretrained_model)
    for k in pretrained_model.keys():
        if k not in model.state_dict().keys():
            del pretrained_modeled[k]
    model.load_state_dict(pretrained_modeled)
    model.word_emb = utils.load_word_emb(args.glove_embed_path)


def feed_question(qstr, db_id='concert_singer'):
    template = json.load(open("preprocess/fuck.json"))[0]
    template['db_id'] = db_id
    template['question'] = qstr
    template['question_toks'] = nltk.tokenize.TweetTokenizer().tokenize(qstr)
    
    tempdir = tempfile.mkdtemp('lalala')
    print('cp -r fdata ' + tempdir)
    os.system('cp -r fdata ' + tempdir)
    json.dump([template], open(os.path.join(tempdir, "tmp.json"), "w"))
    
    print('preprocess/run_me.sh {} tables.json {}'.format(os.path.join(tempdir, "tmp.json"), os.path.join(tempdir, "fdata", "dev.json")))
    os.system('preprocess/run_me.sh {} tables.json {}'.format(os.path.join(tempdir, "tmp.json"), os.path.join(tempdir, "fdata", "dev.json")))
    return tempdir

    
def do_prediction(db_id='concert_singer'):
    print(sql_data)
    perm = list(range(len(sql_data)))
    examples = utils.to_batch_seq(sql_data, table_data, perm, 0, 1, is_train=False)
    example=examples[0]
    print(example.src_sent)
    results = model.parse(example, args.beam_size)[0]
    sql_sentences = []
    for result in results:
        modified_example = {'model_result_replace': ' '.join([str(x) for x in results[0].actions]), **sql_data[0]}
        sql_sentence = sem2SQL.transform(modified_example, schemas[db_id])[0]
        sql_sentences.append(sql_sentence)
    print(sql_sentences)
    return sql_sentences

In [2]:
# load_model()

Use Column Pointer:  True
load pretrained model from ./saved_model/IRNet_pretrained.model
Loading word embedding from ./data/glove.42B.300d.txt


In [11]:
def e2e_demo(qstr, db_id='concert_singer'):
    tempdir = feed_question(qstr, db_id=db_id)
    load_data(tempdir)
    ret = do_prediction(db_id=db_id)
    shutil.rmtree(tempdir)
    return ret

def e2e_execute_demo(qstr, db_id='concert_singer', last=None):
    import sqlalchemy
    import pandas as pd
    sqls = e2e_demo(qstr, db_id=db_id)
    
    for sql in sqls:
        if last is not None and last != '':
            sql = sql.replace('= 1', "= '{}'".format(last))
            print(sql)
        joins = list(re.findall('([^\s]+)\s+AS\s+([^\s]+)\s+JOIN\s+([^\s]+)\s+AS\s+([^\s]+)', sql))
        ssql = re.split('[^\s]+\s+AS\s+[^\s]+\s+JOIN\s+[^\s]+\s+AS\s+[^\s]+', sql)
        print(ssql, joins)
        assert len(ssql) == len(joins) + 1
        nsql = ''
        
        dbschema = schemas[db_id]
        column_names = dbschema['column_names_original']
        table_names = dbschema['table_names_original']
        fk = dbschema['foreign_keys']
        
        def get_common_key(t1, t2):
            i1 = table_names.index(t1)
            i2 = table_names.index(t2)
            columns_1 = [(i, y) for i, (x, y) in enumerate(column_names) if x == i1]
            columns_2 = [(i, y) for i, (x, y) in enumerate(column_names) if x == i2]
            for _, x in columns_1:
                for _, y in columns_2:
                    if x == y: return x, y
            return None, None
            
        
        for x in range(len(ssql)):
            nsql += ssql[x]
            if x != len(ssql) - 1:
                print(joins[x])
                k1, k2 = get_common_key(joins[x][0], joins[x][2])
                if k1 is not None:
                    nsql += '{0} AS {1} JOIN {2} AS {3} ON {1}.{4} = {3}.{5}'.format(*(list(joins[x]) + [k1, k2]))
                else:
                    nsql += '{0} AS {1} JOIN {2} AS {3}'.format(*joins[x])
        sql = nsql
        
        print(nsql)
        engine = sqlalchemy.engine.create_engine("sqlite:///../text2sql/database/{}/{}.sqlite".format(db_id, db_id))
        df = pd.read_sql(sql, engine)
        if df.shape[0] != 0: return sql, df
    
    return sql, df

In [12]:
model.eval()
e2e_execute_demo("Give me the author of the cheapest book", db_id="book_2")

cp -r fdata /tmp/tmpd06v2aetlalala
preprocess/run_me.sh /tmp/tmpd06v2aetlalala/tmp.json tables.json /tmp/tmpd06v2aetlalala/fdata/dev.json
Loading from datasets...
Loading data from /tmp/tmpd06v2aetlalala/fdata/tables.json
Loading data from /tmp/tmpd06v2aetlalala/fdata/train.json
Loading data from /tmp/tmpd06v2aetlalala/fdata/dev.json
[{'db_id': 'book_2', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'Give me the author of the cheapest book', 'question_toks': ['give', 'me', 'author', 'of', 'cheapest', 'book'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'book id', 'title', 'issues', 'writer'], 'table_names': ['publication', 'book'], 'col_set': ['*', '

('SELECT T1.Title FROM book AS T1 JOIN publication AS T2 ON T1.Book_ID = T2.Book_ID    ORDER BY T2.Price ASC LIMIT 1    ',
                            Title
 0  Michael Moorcock's Multiverse)

In [13]:
e2e_execute_demo("show me the titles of the books", db_id="book_2")

cp -r fdata /tmp/tmpmhqb73lblalala
preprocess/run_me.sh /tmp/tmpmhqb73lblalala/tmp.json tables.json /tmp/tmpmhqb73lblalala/fdata/dev.json
Loading from datasets...
Loading data from /tmp/tmpmhqb73lblalala/fdata/tables.json
Loading data from /tmp/tmpmhqb73lblalala/fdata/train.json
Loading data from /tmp/tmpmhqb73lblalala/fdata/dev.json
[{'db_id': 'book_2', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'show me the titles of the books', 'question_toks': ['show', 'me', 'title', 'of', 'book'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'book id', 'title', 'issues', 'writer'], 'table_names': ['publication', 'book'], 'col_set': ['*', 'publication id', 'boo

('SELECT T1.Title FROM book AS T1        ',                            Title
 0                 The Black Lamb
 1                    Bloody Mary
 2     Bloody Mary : Lady Liberty
 3                      BrainBanx
 4                      Cyberella
 5                     Dead Corps
 6          The Dome: Ground Zero
 7                   Gemini Blood
 8  Michael Moorcock's Multiverse
 9                    Sheva's War)

In [ ]:
from flask import Flask
from flask import request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/query')
def query():
    q = request.args.get('q')
    db = request.args.get('db')
#     last = request.args.get('last', '')
    
    sql, val = e2e_execute_demo(q, db_id=db, last='')
    try:
        return jsonify({
            "data": json.loads(val.to_json(orient='records')),
            "sql": sql
        })
    except:
        return jsonify({
            "data": [],
            "sql": ''
        })

app.run(host='0.0.0.0', port=443)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:443/ (Press CTRL+C to quit)
119.3.148.45 - - [16/Jan/2020 05:46:40] "GET / HTTP/1.0" 200 -
119.3.148.45 - - [16/Jan/2020 05:46:56] "GET //query?q=fuck%20you&db=college_1 HTTP/1.0" 308 -
119.3.148.45 - - [16/Jan/2020 05:46:56] "GET /?q=fuck%20you&db=college_1 HTTP/1.0" 200 -
119.3.148.45 - - [16/Jan/2020 05:48:00] "GET /text2sql?q=fuck%20you&db=college_1 HTTP/1.0" 404 -
119.3.148.45 - - [16/Jan/2020 05:48:09] "GET /text2sql/query?q=fuck%20you&db=college_1 HTTP/1.0" 404 -


cp -r fdata /tmp/tmpq7myxmislalala
preprocess/run_me.sh /tmp/tmpq7myxmislalala/tmp.json tables.json /tmp/tmpq7myxmislalala/fdata/dev.json
Loading from datasets...
Loading data from /tmp/tmpq7myxmislalala/fdata/tables.json
Loading data from /tmp/tmpq7myxmislalala/fdata/train.json
Loading data from /tmp/tmpq7myxmislalala/fdata/dev.json
[{'db_id': 'college_1', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'fuck you', 'question_toks': ['fuck', 'you'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'class code', 'course code', 'class section', 'class time', 'class room', 'professor employee number', 'course code', 'department code', 'course description', 'course credit', 'department code', 'department name', 'school code', 'emp

119.3.148.45 - - [16/Jan/2020 05:49:30] "GET /query?q=fuck%20you&db=college_1 HTTP/1.0" 200 -


['SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFESSOR AS T1 JOIN EMPLOYEE AS T2 JOIN STUDENT AS T3 WHERE T2.EMP_FNAME = 1       ', 'SELECT T1.PROF_HIGH_DEGREE FROM PROFES

...

Loading from datasets...
Loading data from /tmp/tmps5209gp_lalala/fdata/tables.json
Loading data from /tmp/tmps5209gp_lalala/fdata/train.json
Loading data from /tmp/tmps5209gp_lalala/fdata/dev.json
[{'db_id': 'book_2', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'What is the title of the most expensive book?', 'question_toks': ['what', 'is', 'title', 'of', 'most', 'expensive', 'book', '?'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'book id', 'title', 'issues', 'writer'], 'table_names': ['publication', 'book'], 'col_set': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'title', 'issues', 'writer'], 'col_table': [-1, 0

119.3.148.45 - - [16/Jan/2020 07:07:28] "GET /query?q=What+is+the+title+of+the+most+expensive+book%3F&db=book_2&last= HTTP/1.0" 200 -


Loading from datasets...
Loading data from /tmp/tmpn462vvx5lalala/fdata/tables.json
Loading data from /tmp/tmpn462vvx5lalala/fdata/train.json
Loading data from /tmp/tmpn462vvx5lalala/fdata/dev.json
[{'db_id': 'book_2', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'What is the title of the most expensive book?', 'question_toks': ['what', 'is', 'title', 'of', 'most', 'expensive', 'book', '?'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'book id', 'title', 'issues', 'writer'], 'table_names': ['publication', 'book'], 'col_set': ['*', 'publication id', 'book id', 'publisher', 'publication date', 'price', 'title', 'issues', 'writer'], 'col_table': [-1, 0

119.3.148.45 - - [16/Jan/2020 07:34:25] "GET /query?q=Show+me+the+student+name+with+highest+GPA%3F&db=college_1&last= HTTP/1.0" 200 -


Loading from datasets...
Loading data from /tmp/tmp0mup5rptlalala/fdata/tables.json
Loading data from /tmp/tmp0mup5rptlalala/fdata/train.json
Loading data from /tmp/tmp0mup5rptlalala/fdata/dev.json
[{'db_id': 'college_1', 'query': 'SELECT 1', 'query_toks': ['SELECT', '1'], 'query_toks_no_value': ['select', '1'], 'question': 'Show me the student name with highest GPA?', 'question_toks': ['show', 'me', 'student', 'name', 'with', 'highest', 'gpa', '?'], 'sql': {'except': None, 'from': {'conds': [], 'table_units': [['table_unit', 1]]}, 'groupby': [], 'having': [], 'intersect': None, 'limit': None, 'orderby': [], 'select': [False, [[3, [0, [0, 0, False], None]]]], 'union': None, 'where': []}, 'names': ['*', 'class code', 'course code', 'class section', 'class time', 'class room', 'professor employee number', 'course code', 'department code', 'course description', 'course credit', 'department code', 'department name', 'school code', 'employee number', 'department address', 'department extens